In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

import csv
import time

#website = 'https://www.cars24.com/buy-used-maruti-cars-mumbai/' # not strictly needed for driver.get() but useful for context

csv_file_name = 'cars24_maruti_suzuki_mumbai_all_vehicles.csv'

csv_headers = [
    'Name',
    'Kilometers Driven',
    'Year of Manufacture',
    'Fuel Type',
    'Transmission Type',
    'Price',
    'Location',
    'Owners'
]

print("Attempting to connect to manually launched Chrome instance...")

options = webdriver.ChromeOptions()
# specify debugger address of the chrome instance
options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
# options.add_argument('--headless')
# options.add_argument('--disable-gpu')
# options.add_argument('--no-sandbox')
# options.add_argument('--window-size=1920,1080')
# options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

try:

    driver = webdriver.Chrome(options=options)
    print("Successfully connected to the existing Chrome instance.")


    try:
        print("Verifying initial car listings are present on the manually loaded page...")

        WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.CLASS_NAME, "styles_normalCardWrapper__qDZjq"))
        )
        print("Initial car listings detected. Starting scroll process.")
    except TimeoutException:
        print("Timeout waiting for initial car listings on manually loaded page. Check your manual load or selector.")
        raise

    # Handle Infinite Scroll / Dynamic Content Loading
    scroll_pause_time = 7
    last_height = driver.execute_script("return document.body.scrollHeight")
    scroll_attempts = 0
    max_scroll_attempts = 30

    print(f"Scrolling to load all car listings with a {scroll_pause_time}-second pause...")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            print("Reached end of scrollable content or no more new content loading.")
            break
        last_height = new_height
        scroll_attempts += 1
        print(f"  - Scroll attempt {scroll_attempts}. Current page height: {new_height}px")

        if scroll_attempts >= max_scroll_attempts:
            print(f"Reached maximum scroll attempts ({max_scroll_attempts}). Stopping scrolling.")
            break


    print("Page fully loaded. Extracting content with BeautifulSoup...")
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Parsing with BeautifulSoup and Writing to CSV
    with open(csv_file_name, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(csv_headers)

        car_listings = soup.find_all("div", {"class": "styles_normalCardWrapper__qDZjq"})

        if not car_listings:
            print("No car listings found after dynamic content loading. Check selectors or website structure.")
        else:
            print(f"Found {len(car_listings)} car listings to process.")

        for i, car in enumerate(car_listings):
            try:
                # Name
                name_element = car.find('span', class_='sc-braxZu kjFjan')
                name = name_element.text.strip() if name_element else 'N/A'

                # Kilometers Driven, Fuel Type, Transmission Type, Owners (using find_all for consistency)
                detail_ps = car.find_all("p", class_="sc-braxZu kvfdZL")
                km_driven = detail_ps[0].get_text().strip() if len(detail_ps) > 0 else 'N/A'
                fuel_type = detail_ps[1].get_text().strip() if len(detail_ps) > 1 else 'N/A'
                transmission_type = detail_ps[2].get_text().strip() if len(detail_ps) > 2 else 'N/A'
                owners = detail_ps[3].get_text().strip() if len(detail_ps) > 3 else 'N/A'

                # Year of Manufacture
                year_span = car.find("span", class_="sc-braxZu kjFjan")
                year_of_manufacture = year_span.get_text().strip().split()[0] if year_span else 'N/A'

                # Price
                price_element = car.find('p', class_='sc-braxZu cyPhJl')
                price = price_element.get_text() if price_element else 'N/A'

                # Location
                location_element = car.find('p', class_='sc-braxZu lmmumg')
                location = location_element.get_text().strip() if location_element else 'N/A'

                writer.writerow([
                    name,
                    km_driven,
                    year_of_manufacture,
                    fuel_type,
                    transmission_type,
                    price,
                    location,
                    owners
                ])

            except Exception as e:
                print(f"  - Error extracting details for car listing {i+1}: {e}")
                continue

    print(f"\nScraping complete. Data saved to '{csv_file_name}'.")

except Exception as e:
    print(f"An error occurred during the scraping process: {e}")

finally:
    # driver.quit() will close the manually opened browser window.

    if 'driver' in locals() and driver:
        driver.quit()
        print("Browser closed.")

Attempting to connect to manually launched Chrome instance...
Successfully connected to the existing Chrome instance.
Verifying initial car listings are present on the manually loaded page...
Initial car listings detected. Starting scroll process.
Scrolling to load all car listings with a 7-second pause...
Reached end of scrollable content or no more new content loading.
Page fully loaded. Extracting content with BeautifulSoup...
Found 74 car listings to process.

Scraping complete. Data saved to 'cars24_maruti_suzuki_mumbai_all_vehicles.csv'.
Browser closed.
